In [ ]:
# ============================================================================
# AUTOMATIZACIÓN CON SCRIPTS - NIVEL INTERMEDIO
# ============================================================================

#crear_banner("AUTOMATIZACIÓN CON SCRIPTS DE PYTHON")

print(" **Objetivo:** Crear scripts que se ejecuten automáticamente")
print(" **Aprenderemos:**")
print("   • Crear funciones reutilizables")
print("   • Manejar errores")
print("   • Logging (registros)")
print("   • Configuración externa")
print()

# ============================================================================
# SCRIPT 1: PROCESADOR DE ARCHIVOS AUTOMÁTICO
# ============================================================================

print(" **SCRIPT 1: Procesador automático de archivos**")
print("Este script busca archivos nuevos y los procesa automáticamente")
print()

import os
import pandas as pd
import logging
from datetime import datetime
import json

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('etl_process.log'),
        logging.StreamHandler()
    ]
)

class ProcesadorETL:
    """Clase para automatizar procesos ETL"""

    def __init__(self, config_file='config.json'):
        """Inicializar con configuración"""
        self.logger = logging.getLogger(__name__)
        self.config = self.cargar_configuracion(config_file)


    def cargar_configuracion(self, config_file):
        """Cargar configuración desde archivo JSON"""
        config_default = {
            "carpeta_entrada": "datos_entrada",
            "carpeta_salida": "datos_procesados",
            "carpeta_backup": "backup",
            "formatos_permitidos": [".csv", ".xlsx", ".json"],
            "separador_csv": ",",
            "encoding": "utf-8"
        }

        try:
            if os.path.exists(config_file):
                with open(config_file, 'r') as f:
                    config = json.load(f)
                self.logger.info(f"Configuración cargada desde {config_file}")
            else:
                config = config_default
                # Crear archivo de configuración
                with open(config_file, 'w') as f:
                    json.dump(config_default, f, indent=4)
                self.logger.info(f"Archivo de configuración creado: {config_file}")
        except Exception as e:
            self.logger.error(f"Error cargando configuración: {e}")
            config = config_default

        return config

    def crear_carpetas(self):
        """Crear carpetas necesarias si no existen"""
        carpetas = [
            self.config['carpeta_entrada'],
            self.config['carpeta_salida'],
            self.config['carpeta_backup']
        ]

        for carpeta in carpetas:
            if not os.path.exists(carpeta):
                os.makedirs(carpeta)
                self.logger.info(f"Carpeta creada: {carpeta}")

    def buscar_archivos_nuevos(self):
        """Buscar archivos para procesar"""
        carpeta = self.config['carpeta_entrada']
        formatos = self.config['formatos_permitidos']

        archivos = []
        if os.path.exists(carpeta):
            for archivo in os.listdir(carpeta):
                if any(archivo.endswith(fmt) for fmt in formatos):
                    ruta_completa = os.path.join(carpeta, archivo)
                    archivos.append(ruta_completa)

        self.logger.info(f"Encontrados {len(archivos)} archivos para procesar")
        return archivos

    def extraer_datos(self, archivo):
        """Extraer datos según el tipo de archivo"""
        try:
            extension = os.path.splitext(archivo)[1].lower()

            if extension == '.csv':
                df = pd.read_csv(archivo,
                               sep=self.config['separador_csv'],
                               encoding=self.config['encoding'])
            elif extension == '.xlsx':
                df = pd.read_excel(archivo)
            elif extension == '.json':
                df = pd.read_json(archivo)
            else:
                raise ValueError(f"Formato no soportado: {extension}")

            self.logger.info(f"Datos extraídos de {archivo}: {len(df)} filas")
            return df

        except Exception as e:
            self.logger.error(f"Error extrayendo datos de {archivo}: {e}")
            return None

    def transformar_datos(self, df, nombre_archivo):
        """Aplicar transformaciones estándar"""
        try:
            df_transformado = df.copy()

            # 1. Limpiar espacios en columnas de texto
            for col in df_transformado.select_dtypes(include=['object']).columns:
                df_transformado[col] = df_transformado[col].astype(str).str.strip()

            # 2. Convertir fechas si existen columnas con 'fecha' en el nombre
            fecha_cols = [col for col in df_transformado.columns if 'fecha' in col.lower()]
            for col in fecha_cols:
                df_transformado[col] = pd.to_datetime(df_transformado[col], errors='coerce')

            # 3. Agregar metadatos
            df_transformado['archivo_origen'] = nombre_archivo
            df_transformado['fecha_procesamiento'] = datetime.now()

            # 4. Eliminar duplicados
            filas_antes = len(df_transformado)
            df_transformado = df_transformado.drop_duplicates()
            filas_despues = len(df_transformado)

            if filas_antes != filas_despues:
                self.logger.info(f"Eliminados {filas_antes - filas_despues} duplicados")

            self.logger.info(f"Datos transformados: {len(df_transformado)} filas finales")
            return df_transformado

        except Exception as e:
            self.logger.error(f"Error transformando datos: {e}")
            return None

    def cargar_datos(self, df, nombre_archivo):
        """Guardar datos procesados"""
        try:
            # Crear nombre de archivo de salida
            nombre_base = os.path.splitext(os.path.basename(nombre_archivo))[0]
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            nombre_salida = f"{nombre_base}_procesado_{timestamp}.csv"

            ruta_salida = os.path.join(self.config['carpeta_salida'], nombre_salida)

            # Guardar datos
            df.to_csv(ruta_salida, index=False, encoding=self.config['encoding'])

            self.logger.info(f"Datos guardados en: {ruta_salida}")
            return ruta_salida

        except Exception as e:
            self.logger.error(f"Error guardando datos: {e}")
            return None

    def mover_a_backup(self, archivo):
        """Mover archivo procesado a backup"""
        try:
            nombre_archivo = os.path.basename(archivo)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            nombre_backup = f"{timestamp}_{nombre_archivo}"

            ruta_backup = os.path.join(self.config['carpeta_backup'], nombre_backup)

            # Mover archivo
            os.rename(archivo, ruta_backup)
            self.logger.info(f"Archivo movido a backup: {ruta_backup}")

        except Exception as e:
            self.logger.error(f"Error moviendo archivo a backup: {e}")

    def procesar_archivo(self, archivo):
        """Procesar un archivo completo (ETL)"""
        self.logger.info(f"Iniciando procesamiento de: {archivo}")

        # EXTRACT
        df = self.extraer_datos(archivo)
        if df is None:
            return False

        # TRANSFORM
        df_transformado = self.transformar_datos(df, archivo)
        if df_transformado is None:
            return False

        # LOAD
        archivo_salida = self.cargar_datos(df_transformado, archivo)
        if archivo_salida is None:
            return False

        # Mover a backup
        self.mover_a_backup(archivo)

        self.logger.info(f"Procesamiento completado exitosamente: {archivo}")
        return True

    def ejecutar_proceso_completo(self):
        """Ejecutar proceso ETL completo"""
        self.logger.info("=== INICIANDO PROCESO ETL AUTOMÁTICO ===")

        # Crear carpetas necesarias
        self.crear_carpetas()

        # Buscar archivos
        archivos = self.buscar_archivos_nuevos()

        if not archivos:
            self.logger.info("No hay archivos para procesar")
            return

        # Procesar cada archivo
        exitosos = 0
        fallidos = 0

        for archivo in archivos:
            if self.procesar_archivo(archivo):
                exitosos += 1
            else:
                fallidos += 1

        self.logger.info(f"=== PROCESO COMPLETADO ===")
        self.logger.info(f"Archivos procesados exitosamente: {exitosos}")
        self.logger.info(f"Archivos con errores: {fallidos}")

# Demostración del procesador
print(" **Creando y ejecutando el procesador ETL automático:**")

# Crear datos de ejemplo para demostrar
os.makedirs('datos_entrada', exist_ok=True)

# Crear archivo CSV de ejemplo
datos_ejemplo = pd.DataFrame({
    'fecha': ['2024-01-15', '2024-01-16', '2024-01-17'],
    'producto': ['  Laptop  ', 'MOUSE', 'teclado  '],
    'precio': [1200.50, 25.99, 45.0],
    'cantidad': [1, 2, 1],
    'vendedor': ['Ana García', 'Luis Martín', 'María López']
})

datos_ejemplo.to_csv('datos_entrada/ventas_ejemplo.csv', index=False)
print(" Archivo de ejemplo creado: datos_entrada/ventas_ejemplo.csv")

# Ejecutar el procesador
procesador = ProcesadorETL()
procesador.ejecutar_proceso_completo()

print("\n **Archivos generados:**")
print("    config.json - Configuración del procesador")
print("    etl_process.log - Log de ejecución")
print("    datos_procesados/ - Archivos procesados")
print("    backup/ - Archivos originales respaldados")